In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse the XML file
tree = ET.parse("021525.xml")
root = tree.getroot()

# Create empty lists to hold data for DataFrames
hosts_data = []
ports_data = []
scans_data = []

# Extract scan metadata
scan_command = root.attrib.get('args', 'Unknown')
scans_data.append({'scan_command': scan_command})

# Process each host in the XML file
for host in root.findall("host"):
    # Extract host information
    ip_address = None
    mac_address = None
    hostname = None
    os = None
    status = None

    # IP address
    ip_elem = host.find("address[@addrtype='ipv4']")
    if ip_elem is not None:
        ip_address = ip_elem.attrib['addr']

    # MAC address
    mac_elem = host.find("address[@addrtype='mac']")
    if mac_elem is not None:
        mac_address = mac_elem.attrib['addr']

    # Hostname
    hostname_elem = host.find("hostnames/hostname")
    if hostname_elem is not None:
        hostname = hostname_elem.attrib.get('name', None)

    # Operating System
    os_elem = host.find("os/osmatch")
    if os_elem is not None:
        os = os_elem.attrib.get('name', None)

    # Status
    status_elem = host.find("status")
    if status_elem is not None:
        status = status_elem.attrib.get('state', None)

    # Append host data to the list
    hosts_data.append({
        'ip_address': ip_address,
        'mac_address': mac_address,
        'hostname': hostname,
        'os': os,
        'status': status
    })

    # Process ports
    for port in host.findall("ports/port"):
        port_number = port.attrib.get('portid', None)
        protocol = port.attrib.get('protocol', None)
        state = None
        service = None
        version = None

        # State
        state_elem = port.find("state")
        if state_elem is not None:
            state = state_elem.attrib.get('state', None)

        # Service
        service_elem = port.find("service")
        if service_elem is not None:
            service = service_elem.attrib.get('name', None)
            version = service_elem.attrib.get('version', None)

        # Append port data to the list
        ports_data.append({
            'ip_address': ip_address,
            'port_number': port_number,
            'protocol': protocol,
            'state': state,
            'service': service,
            'version': version
        })

# Convert lists to Pandas DataFrames
scans_df = pd.DataFrame(scans_data)
hosts_df = pd.DataFrame(hosts_data)
ports_df = pd.DataFrame(ports_data)

# Display the DataFrames
print("Scan Metadata:")
print(scans_df)

print("\nHosts Data:")
print(hosts_df)

print("\nPorts Data:")
print(ports_df)


Scan Metadata:
                     scan_command
0  nmap -sV -O -v 192.168.55.0/24

Hosts Data:
         ip_address        mac_address                     hostname  \
0      192.168.55.0               None                         None   
1      192.168.55.2               None                         None   
2      192.168.55.3               None                         None   
3      192.168.55.4               None                         None   
4      192.168.55.5               None                         None   
..              ...                ...                          ...   
251  192.168.55.250  90:09:D0:14:E5:F7                         None   
252  192.168.55.251  02:42:C0:A8:37:FB                      pi.hole   
253  192.168.55.252  02:42:C0:A8:37:FC                         None   
254  192.168.55.253  02:42:C0:A8:37:FD                         None   
255   192.168.55.10               None  timdesktop.larsonsplace.com   

                                      os status  
0

In [2]:
hosts_df

,ip_address,mac_address,hostname,os,status
0,192.168.55.0,None,None,None,down
1,192.168.55.2,None,None,None,down
2,192.168.55.3,None,None,None,down
3,192.168.55.4,None,None,None,down
4,192.168.55.5,None,None,None,down
...,...,...,...,...,...
251,192.168.55.250,90:09:D0:14:E5:F7,None,Linux 3.2 - 4.14,up
252,192.168.55.251,02:42:C0:A8:37:FB,pi.hole,Linux 3.2 - 4.14,up
253,192.168.55.252,02:42:C0:A8:37:FC,None,Linux 3.2 - 4.14,up
254,192.168.55.253,02:42:C0:A8:37:FD,None,Aruba ArubaOS-CX 10.04,up


In [3]:
with pd.ExcelWriter("nmap_results.xlsx") as writer:
    scans_df.to_excel(writer, sheet_name="Scan Metadata", index=False)
    hosts_df.to_excel(writer, sheet_name="Hosts", index=False)
    ports_df.to_excel(writer, sheet_name="Ports", index=False)


In [4]:
nmap_host_up_df = pd.read_csv("nmap_up_01.csv", encoding="utf-8")


In [5]:
nmap_host_up_df

,ip_address,mac_address,hostname,os,status
0,192.168.55.1,F0:AD:4E:26:D9:54,NaN,FreeBSD 11.2-RELEASE,up
1,192.168.55.12,52:1B:A6:A7:FD:36,NaN,Apple macOS 11 (Big Sur) - 13 (Ventura) or iOS...,up
2,192.168.55.13,84:72:07:7B:20:4F,NaN,NaN,up
3,192.168.55.14,A0:40:A0:67:60:18,NaN,OpenWrt Chaos Calmer 15.05 (Linux 3.18) or Des...,up
4,192.168.55.15,B0:68:E6:ED:C0:8F,NaN,Brother or HP printer (VxWorks),up
5,192.168.55.18,10:27:F5:7A:A9:7E,NaN,lwIP 1.4.1 - 2.0.3,up
6,192.168.55.19,64:52:99:5C:7D:98,NaN,Chamberlain myQ garage door opener,up
7,192.168.55.20,A0:40:A0:67:4C:77,NaN,OpenWrt Chaos Calmer 15.05 (Linux 3.18) or Des...,up
8,192.168.55.22,C8:69:CD:11:67:14,appletv.larsonsplace.com,Apple iOS 14.0 - 15.6 or tvOS 14.3 - 16.1 (Dar...,up
9,192.168.55.24,78:E1:03:45:7C:F5,NaN,Linux 3.2 - 4.14,up


In [5]:

# Get the maximum length of each column
max_lengths = hosts_df.apply(lambda col: col.astype(str).map(len)).max()

# Display the results
print(max_lengths)

ip_address     14
mac_address    17
hostname       29
os             79
status          4
dtype: int64


In [6]:
import requests

In [7]:
# Convert DataFrame to JSON
hosts_json = hosts_df.to_dict(orient="records")

# Define the FastAPI endpoint URL (replace with your server's IP)
fastapi_url = "http://127.0.0.1:8000/nmap/devices/"

# Send a POST request with the JSON data
response = requests.post(fastapi_url, json=hosts_json)

# Print response
print(response.status_code, response.json())

422 {'detail': [{'type': 'string_type', 'loc': ['body', 0, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 1, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 2, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 3, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 4, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 5, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 6, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 7, 'mac_address'], 'msg': 'Input should be a valid string', 'input': None}, {'type': 'string_type', 'loc': ['body', 8, 'mac_address'

In [8]:
import pandas as pd
import requests

# Example: Load your hosts_df (replace with actual data)
hosts_df = pd.DataFrame([
    {"ip_address": "192.168.1.1", "mac_address": "00:1A:2B:3C:4D:5E", "hostname": "router", "os": "Linux", "status": "up"},
    {"ip_address": "192.168.1.100", "mac_address": None, "hostname": "laptop", "os": "Windows", "status": "up"},
])

# Fill missing MAC addresses with an empty string or a default value
hosts_df["mac_address"] = hosts_df["mac_address"].fillna("00:00:00:00:00:00")

# Convert DataFrame to JSON
hosts_json = hosts_df.to_dict(orient="records")

# Define the FastAPI endpoint URL (replace with your server's IP)
fastapi_url = "http://127.0.0.1:8000/nmap/devices/"

# Send a POST request with the JSON data
response = requests.post(fastapi_url, json=hosts_json)

# Print response
print(response.status_code, response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import requests
response = requests.get("http://localhost:8000/nmap/devices/")
print(response.status_code, response.text)


200 []


In [10]:
response = requests.post(fastapi_url, json=hosts_json)

# Print raw response before attempting to decode JSON
print("Status Code:", response.status_code)
print("Response Text:", response.text)

# Try to parse JSON only if response is not empty
if response.text.strip():
    print(response.json())
else:
    print("Empty response received from server.")


Status Code: 500
Response Text: Internal Server Error


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
test_device = [
    {
        "ip_address": "192.168.1.1",
        "mac_address": "00:1A:2B:3C:4D:5E",
        "hostname": "router",
        "os": "Linux",
        "status": "up"
    }
]

response = requests.post(fastapi_url, json=test_device)
print(response.status_code, response.text)


200 [{"ip_address":"192.168.1.1","mac_address":"00:1A:2B:3C:4D:5E","hostname":"router","os":"Linux","status":"up","id":2,"uploaded_at":"2025-02-19T06:50:00.590427"}]
